In [1]:
import cv2
import numpy as np
from pathlib import Path
import yaml

from hfnet.datasets.cmu import Cmu
from hfnet.evaluation.localization import Localization
from hfnet.evaluation.utils.db_management import read_query_list
from hfnet.evaluation.loaders import export_loader
from hfnet.settings import DATA_PATH, EXPER_PATH

from utils import plot_matches

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
slice_ = 'slice2'

# Run one of the three configs

### NV+SP

In [3]:
config_global = {
    'db_name': 'globaldb_netvlad.pkl',
    'experiment': 'netvlad/cmu/'+slice_,
    'predictor': export_loader, 
    'has_keypoints': False, 
    'has_descriptors': False, 
    'pca_dim': 1024,
    'num_prior': 10,
}
config_local = {
    'db_name': 'localdb_superpoint.pkl',
    'experiment': 'superpoint/cmu/'+slice_,
    'predictor': export_loader,
    'has_keypoints': True,
    'has_descriptors': True,
    'binarize': False,
#    'do_nms': True,
#    'nms_thresh': 4,
    'num_features': 2000,
    'ratio_thresh': 0.9,
}
model = 'sp_model'

### HF-Net

In [11]:
config_global = {
    'db_name': 'globaldb_hfnet.pkl',
    'experiment': 'hfnet/cmu/'+slice_,
    'predictor': export_loader, 
    'has_keypoints': False, 
    'has_descriptors': False, 
    'pca_dim': 1024,
    'num_prior': 10,
}
config_local = {
    'db_name': 'localdb_hfnet.pkl',
    'experiment': 'hfnet/cmu/'+slice_,
    'predictor': export_loader,
    'has_keypoints': True,
    'has_descriptors': True,
    'binarize': False,
#    'do_nms': True,
#    'nms_thresh': 4,
    'num_features': 2000,
    'ratio_thresh': 0.9,
}
model = 'sp_model'

### NV+SIFT

In [ ]:
config_global = {
    'db_name': 'globaldb_netvlad_sift-retry.pkl',
    'experiment': 'netvlad/cmu_resize-1024/'+slice_,
    'predictor': export_loader, 
    'has_keypoints': False, 
    'has_descriptors': False, 
    'pca_dim': 1024,
    'num_prior': 10,
}
config_local = {
    'db_name': 'localdb_sift.pkl',
    'colmap_db': 'colmapdb_sift_database.db',
    'colmap_db_queries': 'colmapdb_sift_queries.db',
    'broken_paths': True,
    'ratio_thresh': 0.7,
}
model = 'sift_model'

# Setup localization

In [12]:
config_pose = {
    'reproj_error': 5,
    'min_inliers': 12,
}
config = {'global': config_global, 'local': config_local, 'pose': config_pose}
loc = Localization('cmu/'+slice_, model, config)
queries = read_query_list(Path(loc.base_path, slice_+'.queries_with_intrinsics.txt'))

[04/07/2019 13:44:30 INFO] Importing COLMAP model sp_model
[04/07/2019 13:44:31 INFO] Number of images: 468
Number of points: 47560
Median keypoints per image: 1319.0
Ratio of matched keypoints: 0.376

[04/07/2019 13:44:31 INFO] Importing global and local databases
[04/07/2019 13:44:32 INFO] Indexing descriptors


# Optionally: isolate successful or failed queries

In [ ]:
eval_file = f'cmu/eval_name_{slice_}.yaml'
with open(Path(EXPER_PATH, 'eval', eval_file), 'r') as f:
    failures = yaml.load(f)['metrics']['failure']
#queries = [queries[f] for f in failures]  # failures
queries = [queries[i] for i in range(len(queries)) if i not in set(failures)]  # success

# Ready query dataset

In [13]:
np.random.RandomState(0).shuffle(queries)
query_dataset = Cmu(**{'resize_max': 1024,
                       'image_names': [q.name for q in queries], 'prefix': slice_})
def get_image(name):
    path = Path(DATA_PATH, query_dataset.dataset_folder, slice_, name)
    return cv2.imread(path.as_posix())[..., ::-1]

# Localize

In [ ]:
query_iter = query_dataset.get_test_set()
for i, query_info, query_data in zip(range(5), queries, query_iter):
    results, debug = loc.localize(query_info, query_data, debug=True)
    s = f'{i} {"Success" if results.success else "Failure"}, inliers {results.num_inliers:^4}, ' \
        + f'ratio {results.inlier_ratio:.3f}, landmarks {len(debug["matching"]["lm_frames"]):>4}, ' \
        + f'spl {debug["index_success"]:>2}, places {[len(p) for p in debug["places"]]:}, ' \
        + f'pos {[f"{n:.1f}" for n in results.T[:3, 3]]}'
    print(s)
    
    sorted_frames, counts = np.unique(
        [debug['matching']['lm_frames'][m2] for m1, m2 in debug['matches'][debug['inliers']]],
        return_counts=True)
    best_id = sorted_frames[np.argmax(counts)]

    query_image = get_image(query_info.name)
    best_image = get_image(loc.images[best_id].name)
    best_matches_inliers = [(m1, debug['matching']['lm_indices'][m2]) 
                            for m1, m2 in debug['matches'][debug['inliers']] 
                            if debug['matching']['lm_frames'][m2] == best_id]
    
    plot_matches(
        query_image, debug['query_item'].keypoints,
        best_image, loc.local_db[best_id].keypoints,
        np.array(best_matches_inliers), color=(0, 1., 0),
        dpi=100, ylabel=str(i), thickness=1.)